 # Случайные леса
 __Суммарное количество баллов: 10__

 В этом задании вам предстоит реализовать ансамбль деревьев решений, известный как случайный лес, применить его к публичным данным и сравнить его эффективность с ансамблем, предоставляемым библиотекой CatBoost.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import matplotlib
import copy
from catboost import CatBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from scipy import stats
from sklearn.model_selection import cross_validate
from sklearn.base import BaseEstimator

In [3]:
def read_cancer_dataset(path_to_csv):
    df = pd.read_csv(path_to_csv)
    df = df.sample(frac=1).reset_index(drop=True)
    X = df.loc[:, df.columns != 'label']
    y = df.label.astype('category')
    y = df.label.apply(lambda x: 1 if x=='M' else 0)
    return (X,y)

def read_spam_dataset(path_to_csv):
    df = pd.read_csv(path_to_csv)
    df = df.sample(frac=1).reset_index(drop=True)
    df.capital_run_length_average = (df.capital_run_length_average-df.capital_run_length_average.mean())/df.capital_run_length_average.std()
    df.capital_run_length_longest = (df.capital_run_length_longest-df.capital_run_length_longest.mean())/df.capital_run_length_longest.std()
    df.capital_run_length_total = (df.capital_run_length_total-df.capital_run_length_total.mean())/df.capital_run_length_total.std()
    X = df.loc[:, df.columns != 'label']
    y = df.label
    return (X,y)


 ### Задание 1 (3 балла)
 Реализуем сам Random Forest. Идея очень простая: строим `n` деревьев, а затем берем модальное предсказание.

 #### Параметры конструктора
 `n_estimators` - количество используемых для предсказания деревьев.

 Остальное - параметры деревьев.

 #### Методы
 `fit(X, y)` - строит `n_estimators` деревьев по выборке `X`.

 `predict(X)` - для каждого элемента выборки `X` возвращает самый частый класс, который предсказывают для него деревья.

In [5]:
class RandomForestClassifier (BaseEstimator): 

    def __init__(self, criterion="gini", max_depth=None, min_samples_leaf=1, max_features="auto", n_estimators=10):
        self.criterion = criterion
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        self.max_features = max_features
        self.n_estimators = n_estimators

    def fit(self, X, y):
        self.trees = []
        self.X = X
        self.y = y
        for i in range(0, self.n_estimators):
            X_sampled = X.sample(n = len(X), replace=True)
            y_sampled = y[X_sampled.index]
            weakTree = DecisionTreeClassifier(max_depth = self.max_depth, min_samples_leaf = self.min_samples_leaf, max_features = self.max_features)
            fittedTree = weakTree.fit(X_sampled, y_sampled)
            self.trees.append((fittedTree, X_sampled, y_sampled))
        return self.trees
    
    def predict(self, X):
        treesPredictions = list(map(lambda tree: tree[0].predict(X), self.trees))
        modePredictions =  stats.mode(treesPredictions)
        self.mode_sizes = modePredictions[1][0]
        return modePredictions[0][0]

    def predict_proba(self, X):
        preds = self.predict(X)
        probs = np.array(list(zip(1 - preds, preds)))
        return probs

 ### Задание 3 (3 балла)
 Оптимизируйте по AUC на кроссвалидации (размер валидационной выборки - 20%) параметры своей реализации Random Forest:

 максимальную глубину деревьев из [2, 3, 5, 7, 10], количество деревьев из [5, 10, 20, 30, 50, 100].

 Постройте ROC кривую (и выведите AUC и accuracy) для лучшего варианта.

 Подсказка: можно построить сразу 100 деревьев глубины 10, а потом убирать деревья и
 глубину.

In [10]:

X, y = read_cancer_dataset("cancer.csv")

depths = [2,3,5,7,10]
treeNumbers = [5,10,20,30,50,100]

maxScore = -1
maxScoreDepth = -1
maxScoreTreeNumber = -1
accuracy = 0

for depth in depths:
    for treeNumber in treeNumbers:
        forest = RandomForestClassifier(max_depth = depth, n_estimators = treeNumber)
        score = cross_validate(forest, X, y,  scoring='roc_auc')
        score_accuracy = cross_validate(forest, X, y,  scoring='accuracy')

        score = score["test_score"].mean()
        if(score > maxScore):
            maxScore = score
            accuracy = score_accuracy
            maxScoreDepth = forest.max_depth
            maxScoreTreeNumber = forest.n_estimators

print("Лучшая глубина: ", maxScoreDepth, "Лучшее количество деревьев: ", maxScoreTreeNumber)

print("AUC для лучшей глубины: ", maxScore)

forest = RandomForestClassifier(max_depth = maxScoreDepth, n_estimators = maxScoreTreeNumber)
score_accuracy = cross_validate(forest, X, y,  scoring='accuracy')
print("Accuracy для лучшей глубины: ", score_accuracy["test_score"].mean())


Лучшая глубина:  10 Лучшее количество деревьев:  30
AUC для лучшей глубины:  0.9613479367450564
Accuracy для лучшей глубины:  0.9525073746312686


 ### Задание 4 (3 балла)
 Часто хочется понимать, насколько большую роль играет тот или иной признак для предсказания класса объекта. Есть различные способы посчитать его важность. Один из простых способов сделать это для Random Forest выглядит так:
 1. Посчитать out-of-bag ошибку предсказания `err_oob` (https://en.wikipedia.org/wiki/Out-of-bag_error)
 2. Перемешать значения признака `j` у объектов выборки (у каждого из объектов изменится значение признака `j` на какой-то другой)
 3. Посчитать out-of-bag ошибку (`err_oob_j`) еще раз.
 4. Оценкой важности признака `j` для одного дерева будет разность `err_oob_j - err_oob`, важность для всего леса считается как среднее значение важности по деревьям.

 Реализуйте функцию `feature_importance`, которая принимает на вход Random Forest и возвращает массив, в котором содержится важность для каждого признака.

In [13]:
def feature_importance(rfc):

    featureImportances = []

    for feature in rfc.X.columns:

        scoreSum = 0
        for tree, tree_train_X, tree_train_y in rfc.trees:

            X_test = rfc.X.drop(index = tree_train_y)
            y_test = rfc.y[X_test.index]

            prediction = tree.predict(X_test)
            scoreWithFeature = 1 - np.mean(prediction == y_test)

            X_shuffled = pd.DataFrame(X_test)
            np.random.shuffle(X_shuffled.loc[:,feature].values)
            
            prediction = tree.predict(X_shuffled)
            scoreWithoutFeature = 1 - np.mean(prediction == y_test)

            scoreSum += scoreWithoutFeature - scoreWithFeature

        featureImportances.append(scoreSum / rfc.n_estimators)

            #err_oob = tree.predict()
    return np.asarray(featureImportances)

def most_important_features(importance, names, k=20):
    # Выводит названия k самых важных признаков
    idicies = np.argsort(importance)[::-1][:k]
    return np.array(names)[idicies]


 Протестируйте решение на простом синтетическом наборе данных. В результате должна получиться точность `1.0`, наибольшее значение важности должно быть у признака с индексом `4`, признаки с индексами `2` и `3`  должны быть одинаково важны, а остальные признаки - не важны совсем.

In [14]:
def synthetic_dataset(size):
    X = [(np.random.randint(0, 2), np.random.randint(0, 2), i % 6 == 3, 
          i % 6 == 0, i % 3 == 2, np.random.randint(0, 2)) for i in range(size)]
    y = [i % 3 for i in range(size)]
    return np.array(X), np.array(y)

X, y = synthetic_dataset(1000)
X = pd.DataFrame(X)
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
importances = feature_importance(rfc)
print("Importance:", np.round(importances,3))
print("Самые важные признаки: ", most_important_features(importances, X.columns, 3))


Accuracy: 1.0
Importance: [0.    0.    0.217 0.215 0.4   0.   ]
Самые важные признаки:  [4 2 3]


 Проверьте, какие признаки важны для датасетов cancer и spam?

In [15]:
X, y = read_cancer_dataset("cancer.csv")
rfc = RandomForestClassifier(n_estimators=100, min_samples_leaf=2, max_depth = 5)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))
importances = feature_importance(rfc)
print("Importance:", np.round(importances,3))
print("5 самых важных признаков: ", most_important_features(importances, X.columns, 5))



Accuracy: 0.9894551845342706
Importance: [0.013 0.007 0.016 0.031 0.003 0.003 0.029 0.039 0.001 0.001 0.008 0.002
 0.011 0.024 0.001 0.002 0.002 0.003 0.001 0.001 0.071 0.01  0.051 0.058
 0.007 0.006 0.027 0.064 0.003 0.002]
5 самых важных признаков:  ['21' '28' '24' '23' '8']


In [16]:
X, y = read_spam_dataset("spam.csv")
rfc = RandomForestClassifier(n_estimators=100, min_samples_leaf = 2, max_depth = 10)
rfc.fit(X, y)
print("Accuracy:", np.mean(rfc.predict(X) == y))

importances = feature_importance(rfc)
print("Importance:", np.round(importances,3))
print("10 самых важных признаков: ", most_important_features(importances, X.columns, 10))


Accuracy: 0.9595740056509454
Importance: [0.001 0.002 0.004 0.    0.014 0.003 0.04  0.006 0.002 0.003 0.004 0.004
 0.001 0.    0.001 0.033 0.009 0.002 0.011 0.002 0.021 0.001 0.012 0.01
 0.034 0.015 0.019 0.003 0.002 0.003 0.001 0.    0.001 0.    0.002 0.001
 0.008 0.    0.001 0.    0.001 0.003 0.001 0.001 0.004 0.012 0.    0.
 0.001 0.004 0.001 0.046 0.039 0.001 0.029 0.033 0.02 ]
10 самых важных признаков:  ['char_freq_!' 'word_freq_remove' 'char_freq_$' 'word_freq_hp'
 'word_freq_free' 'capital_run_length_longest'
 'capital_run_length_average' 'word_freq_your' 'capital_run_length_total'
 'word_freq_george']


 ### Задание 5 (1 балл)
 В качестве аьтернативы попробуем CatBoost.

 Туториалы можно найти, например, [здесь](https://catboost.ai/docs/) и [здесь](https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb).

 Также, как и реализованный ними RandomForest, примените его для наших датасетов.

In [17]:
X, y = read_cancer_dataset("cancer.csv")
catboost = CatBoostClassifier(iterations=1000, learning_rate=1, logging_level='Silent')
catboost.fit(X,y)
print("Accuracy:", np.mean(catboost.predict(X) == y))

importances = catboost.get_feature_importance()
print("Importance:", np.round(importances,3))
print("5 самых важных признаков: ", most_important_features(importances, X.columns, 5))

Accuracy: 1.0
Importance: [ 6.073  5.58   1.001  1.764  0.886  0.479  0.19   8.045  0.198  1.178
  1.595  0.115  0.284  8.584  1.22   0.408  0.049  0.332  0.106  0.146
  2.285  8.831  0.353 13.825  9.288  1.367 17.725  5.569  0.102  2.425]
5 самых важных признаков:  ['27' '24' '25' '22' '14']


In [18]:
X, y = read_spam_dataset("spam.csv")
catboost = CatBoostClassifier(iterations=1000, learning_rate=1, logging_level='Silent')
catboost.fit(X,y)
print("Accuracy:", np.mean(catboost.predict(X) == y))

importances = catboost.get_feature_importance()
print("Importance:", np.round(importances,3))
print("10 самых важных признаков: ", most_important_features(importances, X.columns, 10))


Accuracy: 0.9993479678330798
Importance: [3.9900e-01 3.6400e-01 1.7970e+00 4.2000e-02 2.4780e+00 1.0990e+00
 6.2180e+00 6.5200e-01 3.3200e-01 7.1600e-01 4.1300e-01 2.1000e+00
 1.5600e-01 3.3700e-01 5.0000e-03 3.1170e+00 1.1050e+00 9.6600e-01
 3.6280e+00 4.7500e-01 2.7440e+00 2.4600e-01 6.5200e-01 6.9800e-01
 6.8970e+00 8.4400e-01 1.3646e+01 4.5700e-01 8.1000e-02 1.3000e-01
 1.0000e-02 0.0000e+00 3.3300e-01 0.0000e+00 3.7600e-01 5.0900e-01
 2.0930e+00 5.9000e-02 6.9400e-01 7.8000e-02 2.2600e-01 2.2990e+00
 6.1000e-02 7.6100e-01 2.4980e+00 3.2000e+00 0.0000e+00 8.5400e-01
 7.0800e-01 2.2000e+00 7.4500e-01 5.5840e+00 5.6380e+00 3.6600e-01
 6.7050e+00 6.0760e+00 5.1300e+00]
10 самых важных признаков:  ['word_freq_george' 'word_freq_hp' 'capital_run_length_average'
 'word_freq_remove' 'capital_run_length_longest' 'char_freq_$'
 'char_freq_!' 'capital_run_length_total' 'word_freq_you' 'word_freq_edu']
